In [1]:
import pickle
import numpy as np

In [5]:
mci_data = pickle.load(open(r'./feature_extraction/MCI_final_features_all_1.pkl', 'rb'))
cn_data = np.array(pickle.load(open(r'./feature_extraction\func_features_AD.pkl', 'rb')))

In [6]:
mci_ad = np.append(mci_data, cn_data, axis=0)

In [8]:
y = np.zeros((mci_data.shape[0],))
y = np.append(y, np.ones((cn_data.shape[0],)))
y.shape

(93,)

In [9]:
mci_ad = np.nan_to_num(mci_ad)

In [10]:
# test data train data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(mci_ad, y, test_size=0.2, random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((74, 13456), (19, 13456), (74,), (19,))

In [11]:
# standard scalar
from sklearn.preprocessing import MinMaxScaler
mms = MinMaxScaler()
X_train_scaled = mms.fit_transform(X_train)
X_test_scaled = mms.transform(X_test)

In [1]:
import torch
from torch import nn

In [12]:
input_size = X_train_scaled.shape[1]
hidden_size = 5000
hidden_size2 = 1000
output_size = 700

In [13]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, random_split

In [14]:
# Model parameters
input_dim = X_train_scaled.shape[1]
encoding_dim = 1024
num_classes = len(set(y_train))  # Assuming y_train contains the class labels
batch_size = 64
epochs = 50
learning_rate = 1e-3

In [15]:

# Define the Autoencoder
class Autoencoder(nn.Module):
    def __init__(self, input_dim, encoding_dim):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, encoding_dim),
            nn.ReLU(),
            nn.Linear(encoding_dim, encoding_dim // 2),
            nn.ReLU()
        )
        self.decoder = nn.Sequential(
            nn.Linear(encoding_dim // 2, encoding_dim),
            nn.ReLU(),
            nn.Linear(encoding_dim, input_dim),
            nn.Sigmoid()
        )
    
    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded, encoded

# Define the Classifier
class Classifier(nn.Module):
    def __init__(self, encoding_dim, num_classes):
        super(Classifier, self).__init__()
        self.classifier = nn.Sequential(
            nn.Linear(encoding_dim // 2, encoding_dim // 4),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(encoding_dim // 4, num_classes),
            nn.Softmax(dim=1)
        )
    
    def forward(self, x):
        return self.classifier(x)


In [16]:

# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)


In [17]:

# Create DataLoader
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)


In [18]:

# Create the Autoencoder
autoencoder = Autoencoder(input_dim=input_dim, encoding_dim=encoding_dim)
criterion = nn.MSELoss()
optimizer = optim.Adam(autoencoder.parameters(), lr=learning_rate)


In [19]:

# Train the Autoencoder
for epoch in range(epochs):
    autoencoder.train()
    train_loss = 0
    for data, _ in train_loader:
        optimizer.zero_grad()
        reconstructed, _ = autoencoder(data)
        loss = criterion(reconstructed, data)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    print(f'Epoch {epoch+1}, Loss: {train_loss / len(train_loader)}')


Epoch 1, Loss: 0.06076010316610336
Epoch 2, Loss: 0.05295880138874054
Epoch 3, Loss: 0.04236038401722908
Epoch 4, Loss: 0.04489137418568134
Epoch 5, Loss: 0.04264094866812229
Epoch 6, Loss: 0.04710724949836731
Epoch 7, Loss: 0.04531878978013992
Epoch 8, Loss: 0.04162396676838398
Epoch 9, Loss: 0.04682427830994129
Epoch 10, Loss: 0.04098748415708542
Epoch 11, Loss: 0.04023466631770134
Epoch 12, Loss: 0.04363626427948475
Epoch 13, Loss: 0.04010681062936783
Epoch 14, Loss: 0.03683747164905071
Epoch 15, Loss: 0.03883066587150097
Epoch 16, Loss: 0.03772849775850773
Epoch 17, Loss: 0.035100373439490795
Epoch 18, Loss: 0.03787737898528576
Epoch 19, Loss: 0.039093900471925735
Epoch 20, Loss: 0.034692125394940376
Epoch 21, Loss: 0.030923940241336823
Epoch 22, Loss: 0.0339828971773386
Epoch 23, Loss: 0.035185279324650764
Epoch 24, Loss: 0.03140402119606733
Epoch 25, Loss: 0.031418521888554096
Epoch 26, Loss: 0.035372812300920486
Epoch 27, Loss: 0.029250643216073513
Epoch 28, Loss: 0.027554603293

In [20]:

# Extract the encoded features
autoencoder.eval()
with torch.no_grad():
    _, encoded_features = autoencoder(X_train_tensor)


In [21]:

# Create the Classifier
classifier = Classifier(encoding_dim=encoding_dim, num_classes=num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(classifier.parameters(), lr=learning_rate)


In [22]:

# Create DataLoader for encoded features
encoded_dataset = TensorDataset(encoded_features, y_train_tensor)
encoded_loader = DataLoader(encoded_dataset, batch_size=batch_size, shuffle=True)


In [23]:

# Train the Classifier
for epoch in range(epochs):
    classifier.train()
    train_loss = 0
    for data, target in encoded_loader:
        optimizer.zero_grad()
        output = classifier(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    print(f'Epoch {epoch+1}, Loss: {train_loss / len(encoded_loader)}')


Epoch 1, Loss: 0.706015944480896
Epoch 2, Loss: 0.7093174755573273
Epoch 3, Loss: 0.6833215355873108
Epoch 4, Loss: 0.6833140850067139
Epoch 5, Loss: 0.6615369617938995
Epoch 6, Loss: 0.6567787826061249
Epoch 7, Loss: 0.6348120868206024
Epoch 8, Loss: 0.632046639919281
Epoch 9, Loss: 0.6645532250404358
Epoch 10, Loss: 0.6854847371578217
Epoch 11, Loss: 0.6195466220378876
Epoch 12, Loss: 0.6367692053318024
Epoch 13, Loss: 0.6818309724330902
Epoch 14, Loss: 0.647771418094635
Epoch 15, Loss: 0.6743091642856598
Epoch 16, Loss: 0.6734460890293121
Epoch 17, Loss: 0.6076741516590118
Epoch 18, Loss: 0.642526239156723
Epoch 19, Loss: 0.6871246993541718
Epoch 20, Loss: 0.630387157201767
Epoch 21, Loss: 0.6183584928512573
Epoch 22, Loss: 0.6069356799125671
Epoch 23, Loss: 0.6266894340515137
Epoch 24, Loss: 0.5970357656478882
Epoch 25, Loss: 0.6535950005054474
Epoch 26, Loss: 0.6264328062534332
Epoch 27, Loss: 0.607656866312027
Epoch 28, Loss: 0.7094417214393616
Epoch 29, Loss: 0.6218211650848389


In [24]:

# Evaluate the Classifier
autoencoder.eval()
classifier.eval()
with torch.no_grad():
    _, encoded_test_features = autoencoder(X_test_tensor)
    output = classifier(encoded_test_features)
    _, predicted = torch.max(output, 1)
    accuracy = (predicted == y_test_tensor).float().mean().item()
    print(f'Test Accuracy: {accuracy}')

Test Accuracy: 0.7894737124443054


In [25]:
# train accuracy
autoencoder.eval()
classifier.eval()
with torch.no_grad():
    _, encoded_test_features = autoencoder(X_train_tensor)
    output = classifier(encoded_test_features)
    _, predicted = torch.max(output, 1)
    accuracy = (predicted == y_train_tensor).float().mean().item()
    print(f'Train Accuracy: {accuracy}')

Train Accuracy: 0.7432432174682617


In [3]:
import pickle
import numpy as np
import os
import regex as re

In [ ]:
# starts with number, ends with ".nii"
re_number_nii = re.compile(r'^\d.*\.nii$')

#starts with number
re_number = re.compile(r'^\d.*')

In [ ]:

            # lovda_img = os.path.join(struct_dir,f)
            # struct_image = nib.load(lovda_img)
            # features = struct_masker.transform(struct_image)
            # correlation_matrix = correlation_measure.fit_transform([features])[0]
            # struct_features.append(correlation_matrix.reshape(-1))
            # print(cnt,f)

1 002_S_1155_seg8.mat
2 003_S_0908_reorient.mat
3 003_S_1122_reorient.mat
4 003_S_6258_reorient.mat
5 003_S_6268_seg8.mat
6 003_S_6432_seg8.mat
7 003_S_6479_reorient.mat
8 003_S_6606_reorient.mat
9 012_S_6073_seg8.mat
10 012_S_6760_seg8.mat
11 013_S_6206_seg8.mat


IndexError: list index out of range